# Метод аддитивной оптимизации

In [1]:
import pandas as pd
import sys
if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO
import numpy as np

## Заведение исходных данных

### Получение весов из работы 1,11 (экспертные методы)  
  
тут обязательно переименовываем столбцы так, чтобы названия соответствовали в этой таблице и в таблице с параметрами

In [2]:
weights = pd.read_csv('Искомый коэффициент значимости.csv',sep=';').set_index('Unnamed: 0').T
weights.rename(columns = {'стоимость (тыс. руб.)' : 'Стоимость, тыс.руб.',
                'площадь (м2)' : 'Площадь, м2',
                'количество комнат' : 'Количество комнат',
                'этаж' : 'Этаж',
                'материал стен': 'Материал'},
              inplace = True)
weights

Unnamed: 0,Район,"Стоимость, тыс.руб.","Площадь, м2",Количество комнат,Этаж,Материал,Сумма
Искомый коэффициент значимости,0.233333,0.22381,0.190476,0.07619,0.085714,0.190476,1.0


### Получение текущих данных

сразу добавляем строку с весами критериев, полученную пунктом выше

In [3]:
data = StringIO('''Адрес;Район;Количество комнат;Площадь, м2;Этаж;Материал;Стоимость, тыс.руб.
Каслинская ул, д. 99в;Калининский;3;100;3/11;Кирпич;8600
Шагольская ул, д. 34;Курчатовский;4;89;4/5;Кирпич;2499
Шагольская ул, д. 41;Курчатовский;3;65;2/5;Панельный;2150
Прокатная ул, д. 26;Металлургический;3;91;1/16;Панельный;2250
Свободы ул, д. 88б;Советский;3;50;6/6;Кирпич;4550
Лобинский 10-й пер, д. 3;Тракторозаводский;3;63;4/5;Панельный;2180
МОПРа площадь ул, д. 9;Центральный;4;198.1;9/16;Кирпич;8000''')
df = pd.read_csv(data, sep=";").set_index('Адрес')
originalDataLen = len(df)
df.loc['Веса критериев'] = weights.iloc[0]
df.head(originalDataLen+10)

,Район,Количество комнат,"Площадь, м2",Этаж,Материал,"Стоимость, тыс.руб."
Адрес,,,,,,
"Каслинская ул, д. 99в",Калининский,3.00000,100.000000,3/11,Кирпич,8600.00000
"Шагольская ул, д. 34",Курчатовский,4.00000,89.000000,4/5,Кирпич,2499.00000
"Шагольская ул, д. 41",Курчатовский,3.00000,65.000000,2/5,Панельный,2150.00000
"Прокатная ул, д. 26",Металлургический,3.00000,91.000000,1/16,Панельный,2250.00000
"Свободы ул, д. 88б",Советский,3.00000,50.000000,6/6,Кирпич,4550.00000
"Лобинский 10-й пер, д. 3",Тракторозаводский,3.00000,63.000000,4/5,Панельный,2180.00000
"МОПРа площадь ул, д. 9",Центральный,4.00000,198.100000,9/16,Кирпич,8000.00000
Веса критериев,0.233333,0.07619,0.190476,0.085714,0.190476,0.22381


### Замена буквенных столбцов на числовые эквиваленты

Тут необходимо определиться насколько важен, например, материал дома. Если кирпичный лучше панельного, то вместо "кирпичный" ставим 1, вместо "панельный" - 2. <b>И не забываем, что будем минимизировать данный столбец в последующем.
    
#### Интересный вопрос возник при оценке этажности. Если параметров много, то какие-то два параметра могут быть ближе друг к другу, чем другие. Например 3/11 и 9/16 не одно и то же, но ближе, чем 9/16 и 6/6. Как соблюсти пропорции при расставлении рангов в таком случае?

In [4]:
data = StringIO('''Мотоцикл;Цена, руб;Год выпуска;Пробег, км;Объем двигателя, см3;Мощность двигателя, л.с. 
BMW R Nine T;1099000.00;2017;1070;1200;125 
Harley-Davidson V-Rod;5300000.00;2015;1;1250;209 
Ducati 1299 Panigale;1399000.00;2016;1600;1299;205 
BMW R 1200 GS;1305000.00;2017;4583;1170;125 
Triumph Tiger Explorer;1270000.00;2018;2900;1200;141
Веса критериев;0.333333333333333;0.0666666666666666;0.24583333333333333333333333;0.1333333333333333333333;0.220833333333333333333333''')
df = pd.read_csv(data, sep=";").set_index('Мотоцикл')
originalDataLen = len(df)-1
df.head(10)

,"Цена, руб",Год выпуска,"Пробег, км","Объем двигателя, см3","Мощность двигателя, л.с."
Мотоцикл,,,,,
BMW R Nine T,1.099000e+06,2017.000000,1070.000000,1200.000000,125.000000
Harley-Davidson V-Rod,5.300000e+06,2015.000000,1.000000,1250.000000,209.000000
Ducati 1299 Panigale,1.399000e+06,2016.000000,1600.000000,1299.000000,205.000000
BMW R 1200 GS,1.305000e+06,2017.000000,4583.000000,1170.000000,125.000000
Triumph Tiger Explorer,1.270000e+06,2018.000000,2900.000000,1200.000000,141.000000
Веса критериев,3.333333e-01,0.066667,0.245833,0.133333,0.220833


In [5]:
df.dtypes

Цена, руб                    float64
Год выпуска                  float64
Пробег, км                   float64
Объем двигателя, см3         float64
Мощность двигателя, л.с.     float64
dtype: object

In [6]:
df.replace(inplace = True,
          to_replace = {
              'Центральный' : 1,
              'Калининский' : 2,
              'Курчатовский' : 3,
              'Металлургический' : 4,
              'Тракторозаводский' : 5,
              'Советский' : 6
          })
df.replace(inplace = True,
          to_replace = {
              'Кирпич' : 1,
              'Панельный' : 2
          })
df.replace(inplace = True,
          to_replace = {
              '3/11' : 1,
              '9/16' : 1,
              '4/5' : 2,
              '4/5' : 2,
              '2/5' : 3,
              '1/16' : 3,
              '6/6' : 4,
          })
# 1-2 этаж я не хочу, будет шум
# высокий этаж без лифта (в 5-6 этажках) я тоже не хочу
# последний этаж тоже не хочу, будет крыша протекать
df

,"Цена, руб",Год выпуска,"Пробег, км","Объем двигателя, см3","Мощность двигателя, л.с."
Мотоцикл,,,,,
BMW R Nine T,1.099000e+06,2017.000000,1070.000000,1200.000000,125.000000
Harley-Davidson V-Rod,5.300000e+06,2015.000000,1.000000,1250.000000,209.000000
Ducati 1299 Panigale,1.399000e+06,2016.000000,1600.000000,1299.000000,205.000000
BMW R 1200 GS,1.305000e+06,2017.000000,4583.000000,1170.000000,125.000000
Triumph Tiger Explorer,1.270000e+06,2018.000000,2900.000000,1200.000000,141.000000
Веса критериев,3.333333e-01,0.066667,0.245833,0.133333,0.220833


## Определим максимум и минимум каждого локального критерия

In [7]:
df.loc['max'] = (df.head(5).max(axis=0))
df.loc['min'] = (df.head(5).min(axis=0))

In [8]:
df.head(3+originalDataLen)

,"Цена, руб",Год выпуска,"Пробег, км","Объем двигателя, см3","Мощность двигателя, л.с."
Мотоцикл,,,,,
BMW R Nine T,1.099000e+06,2017.000000,1070.000000,1200.000000,125.000000
Harley-Davidson V-Rod,5.300000e+06,2015.000000,1.000000,1250.000000,209.000000
Ducati 1299 Panigale,1.399000e+06,2016.000000,1600.000000,1299.000000,205.000000
BMW R 1200 GS,1.305000e+06,2017.000000,4583.000000,1170.000000,125.000000
Triumph Tiger Explorer,1.270000e+06,2018.000000,2900.000000,1200.000000,141.000000
Веса критериев,3.333333e-01,0.066667,0.245833,0.133333,0.220833
max,5.300000e+06,2018.000000,4583.000000,1299.000000,209.000000
min,1.099000e+06,2015.000000,1.000000,1170.000000,125.000000


### Выбор критериев которые максимизируем и минимизируем  
  
Правило выбора таково:  
     1. если критерий имеет прямую зависимость с качеством товара <b>(критерий растет - качество растет), то МАКСИМИЗИРУЕМ</b>  
     2. если критерий имеет обратную зависимость с качеством товара <b>(критерий падает - качество растёт), то МИНИМИЗИРУЕМ</b>  

In [9]:
maximizationMinimizationParametrDict = {}
print('"+" - максимизируем; "-" - минимизируем.')
for columnName in df.columns:
    maximizationMinimizationParametrDict[columnName] = \
        input(f'Критерий: "{columnName}", введите +/-: ')

"+" - максимизируем; "-" - минимизируем.
Критерий: "Цена, руб", введите +/-: -
Критерий: "Год выпуска", введите +/-: -
Критерий: "Пробег, км", введите +/-: -
Критерий: "Объем двигателя, см3", введите +/-: +
Критерий: "Мощность двигателя, л.с. ", введите +/-: +


In [10]:
maximizationMinimizationParametrDict

{'Цена, руб': '-',
 'Год выпуска': '-',
 'Пробег, км': '-',
 'Объем двигателя, см3': '+',
 'Мощность двигателя, л.с. ': '+'}

## Максимизируем/минимизируем столбцы двумя типами

In [11]:
normalizedType1DF = df.copy()
normalizedType2DF = df.copy()
for columnName in df.columns:
    
    denominator = df[columnName]['max'] - df[columnName]['min']
    
    if maximizationMinimizationParametrDict[columnName] == '+':
        normalizedType1DF[columnName].iloc[0:originalDataLen] = \
            df[columnName].iloc[0:originalDataLen] / df[columnName]['max']
        
        normalizedType2DF[columnName].iloc[0:originalDataLen] = \
            (df[columnName].iloc[0:originalDataLen] - df[columnName]['min']) / denominator
    else:
        normalizedType1DF[columnName].iloc[0:originalDataLen] = \
            1 - df[columnName].iloc[0:originalDataLen] / df[columnName]['max']
        
        normalizedType2DF[columnName].iloc[0:originalDataLen] = \
            (df[columnName]['max'] - df[columnName].iloc[0:originalDataLen]) / denominator


### Типом 1: текущее / максимальное; 1 - текущее / максимальное

In [12]:
normalizedType1DF.head(originalDataLen+1)

,"Цена, руб",Год выпуска,"Пробег, км","Объем двигателя, см3","Мощность двигателя, л.с."
Мотоцикл,,,,,
BMW R Nine T,0.792642,0.000496,0.766528,0.923788,0.598086
Harley-Davidson V-Rod,0.000000,0.001487,0.999782,0.962279,1.000000
Ducati 1299 Panigale,0.736038,0.000991,0.650884,1.000000,0.980861
BMW R 1200 GS,0.753774,0.000496,0.000000,0.900693,0.598086
Triumph Tiger Explorer,0.760377,0.000000,0.367227,0.923788,0.674641
Веса критериев,0.333333,0.066667,0.245833,0.133333,0.220833


### Типом 2: ( текущее - мин ) / ( макс - мин ); ( макс - текущее  ) / ( макс - мин ) 

In [13]:
normalizedType2DF.head(originalDataLen)

,"Цена, руб",Год выпуска,"Пробег, км","Объем двигателя, см3","Мощность двигателя, л.с."
Мотоцикл,,,,,
BMW R Nine T,1.000000,0.333333,0.766696,0.232558,0.000000
Harley-Davidson V-Rod,0.000000,1.000000,1.000000,0.620155,1.000000
Ducati 1299 Panigale,0.928588,0.666667,0.651026,1.000000,0.952381
BMW R 1200 GS,0.950964,0.333333,0.000000,0.000000,0.000000
Triumph Tiger Explorer,0.959295,0.000000,0.367307,0.232558,0.190476


In [14]:
normalizedType1DF['Обобщенная функ. цели Fi'] = \
    (normalizedType1DF[:][0:originalDataLen]*normalizedType1DF[:].loc['Веса критериев']).sum(axis=1)
print('Fi max =',normalizedType1DF['Обобщенная функ. цели Fi'].max())

Fi max = 0.7553610850946353


In [15]:
normalizedType1DF.head(originalDataLen+1)

,"Цена, руб",Год выпуска,"Пробег, км","Объем двигателя, см3","Мощность двигателя, л.с.",Обобщенная функ. цели Fi
Мотоцикл,,,,,,
BMW R Nine T,0.792642,0.000496,0.766528,0.923788,0.598086,0.707934
Harley-Davidson V-Rod,0.000000,0.001487,0.999782,0.962279,1.000000,0.595016
Ducati 1299 Panigale,0.736038,0.000991,0.650884,1.000000,0.980861,0.755361
BMW R 1200 GS,0.753774,0.000496,0.000000,0.900693,0.598086,0.503461
Triumph Tiger Explorer,0.760377,0.000000,0.367227,0.923788,0.674641,0.615891
Веса критериев,0.333333,0.066667,0.245833,0.133333,0.220833,NaN


In [16]:
normalizedType2DF['Обобщенная функ. цели Fi'] = \
    (normalizedType2DF[:][0:originalDataLen]*normalizedType2DF[:].loc['Веса критериев']).sum(axis=1)
print('Fi max =',normalizedType2DF['Обобщенная функ. цели Fi'].max())

Fi max = 0.8576685461364975


In [17]:
normalizedType2DF.head(originalDataLen+1)

,"Цена, руб",Год выпуска,"Пробег, км","Объем двигателя, см3","Мощность двигателя, л.с.",Обобщенная функ. цели Fi
Мотоцикл,,,,,,
BMW R Nine T,1.000000,0.333333,0.766696,0.232558,0.000000,0.575043
Harley-Davidson V-Rod,0.000000,1.000000,1.000000,0.620155,1.000000,0.616021
Ducati 1299 Panigale,0.928588,0.666667,0.651026,1.000000,0.952381,0.857669
BMW R 1200 GS,0.950964,0.333333,0.000000,0.000000,0.000000,0.339210
Triumph Tiger Explorer,0.959295,0.000000,0.367307,0.232558,0.190476,0.483133
Веса критериев,0.333333,0.066667,0.245833,0.133333,0.220833,NaN
